In [1]:
import pandas as pd

In [9]:
df=pd.read_json("processed_crypto.json")
df_bitcoin=df[df['asset_name']=='Bitcoin']

In [10]:
df_date_created=df_bitcoin.sort_values('created_utc')
df_date_created['created_utc']=df_date_created['created_utc'].str.slice(0,10)
df_date_created['sma20']=df_date_created['open'].rolling(window=20).mean()
df_date_created['high-low']=df_date_created['sma20'].shift(-1)-df_date_created['sma20']

In [35]:
df_date_created['buy-sell'] = df_date_created['high-low'].apply(lambda x: 0 if x <=0 else 1)


In [37]:
train_x=df_date_created[ (df_date_created['created_utc'].str.slice(0,4)== '2018')]
train_xx=train_x
train_y=train_x['buy-sell']
train_x=train_x.drop(columns=['fin_subheading','news_title','news_url','reddit_title','symbol','tweet_text','tweet_text','asset_name','created_utc','reddit_domain','tweet_hashtags','fin_title','buy-sell',])


9451     1
9452     1
9453     1
9454     1
9455     1
9456     1
9457     1
9458     1
9459     1
9460     1
9461     1
9462     1
9463     1
9464     1
9465     1
9466     1
9467     1
9468     1
9469     1
9470     0
9471     0
9472     0
9473     0
9474     0
9475     0
9476     0
9477     0
9478     0
9479     0
9480     0
        ..
17673    0
17674    1
17675    1
17676    1
17677    1
17678    1
17679    1
17680    1
17681    0
17682    1
17683    0
17684    0
17685    0
17686    0
17687    0
17688    1
17689    0
17690    0
17691    0
17692    0
17693    0
17694    0
17695    0
17696    0
17697    0
17698    0
17699    0
17700    0
17701    0
17702    0
Name: buy-sell, Length: 8252, dtype: int64

In [15]:
test_x=df_date_created[ (df_date_created['created_utc'].str.slice(0,4)== '2019') ]
test_xx=test_x
test_y=test_x['buy-sell']
test_x=test_x.drop(columns=['fin_subheading','news_title','news_url','reddit_title','symbol','tweet_text','tweet_text','asset_name','created_utc','reddit_domain','tweet_hashtags','fin_title','buy-sell'])


In [130]:


import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.utils import to_categorical


model = Sequential()

model.add(Dense(100, activation='relu', input_dim=30))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(1,activation='sigmoid'))

adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy',optimizer=adam, metrics=['accuracy'])


#model.compile(optimizer='adam', loss='mean_squared_error')



model.fit(train_x, train_y,epochs=50,batch_size=20)



Epoch 1/50
8252/8252 [==============================] - 9s 1ms/step - loss: 7.9675 - acc: 0.5030
Epoch 2/50
8252/8252 [==============================] - 3s 387us/step - loss: 7.7719 - acc: 0.5178
Epoch 3/50
8252/8252 [==============================] - 3s 396us/step - loss: 7.7777 - acc: 0.5175
Epoch 4/50
8252/8252 [==============================] - 3s 414us/step - loss: 7.7739 - acc: 0.5177
Epoch 5/50
8252/8252 [==============================] - 3s 411us/step - loss: 7.7739 - acc: 0.5177
Epoch 6/50
8252/8252 [==============================] - 3s 417us/step - loss: 7.7739 - acc: 0.5177
Epoch 7/50
8252/8252 [==============================] - 5s 630us/step - loss: 7.7739 - acc: 0.5177
Epoch 8/50
8252/8252 [==============================] - 5s 625us/step - loss: 7.7739 - acc: 0.5177
Epoch 9/50
8252/8252 [==============================] - 3s 352us/step - loss: 7.7739 - acc: 0.5177
Epoch 10/50
8252/8252 [==============================] - 5s 570us/step - loss: 7.7739 - acc: 0.5177
Epoch 11/50

In [131]:
predict=model.predict(test_x)


In [132]:
from sklearn.metrics import classification_report

In [133]:
print(classification_report(list(test_y),predict))

              precision    recall  f1-score   support

           0       0.52      1.00      0.68       618
           1       0.00      0.00      0.00       581

   micro avg       0.52      0.52      0.52      1199
   macro avg       0.26      0.50      0.34      1199
weighted avg       0.27      0.52      0.35      1199



In [134]:
from sklearn.metrics import accuracy_score

In [135]:
print(accuracy_score(list(test_y),predict))

0.5154295246038365
